# Stochastic stranding
__Суммарное количество баллов: 8__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[ML][HW11] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

По бескрайним каменным джунглям от заказа к заказу бродят курьеры. Их задача - как можно быстрее доставить все заказы, чтобы взять новые. Ничто не может заставить их покинуть вечный цикл доставки.

In [1]:
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
import matplotlib
import copy

from itertools import permutations
from tqdm.notebook import tqdm

In [2]:
def get_colors(distances, alpha=True):
    normed = np.array(distances) - np.min(distances)
    normed = normed / np.max(normed)
    alpha = 1/np.mean([len(row) for row in normed])**0.5 if alpha else 1.
    return [[((1. if n > 0.5 else 2 * n), (1. if n < 0.5 else 2 - 2*n), 0., alpha) 
             for n in row] for row in normed]

def get_coords(points):
    results = []
    for pts in points:
        x_prev, _ = pts[0]
        result = [pts[0]]
        for x, y in list(pts[1:]) + [pts[0]]:
            result.append((x_prev, y))
            result.append((x, y))
            x_prev = x
        results.append(list(zip(*result)))
    return results

def init_figure(X):
    upper_bound, lower_bound = X.max(axis=0) + 1, X.min(axis=0) - 1
    fig, ax = plt.subplots(figsize=(10, 10), nrows=1, ncols=1)
    #ax.set_facecolor((0.1, 0.1, 0.1))
    ax.grid(True)
    #ax.grid(True, color=(0.9, 0.9, 0.9))
    ax.set_xticks(range(lower_bound[0], upper_bound[0]))
    ax.set_yticks(range(lower_bound[1], upper_bound[1]))
    ax.set_xlim(lower_bound[0], upper_bound[0])
    ax.set_ylim(lower_bound[1], upper_bound[1])
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.scatter(X[:, 0], X[:, 1], color=(0, 0, 1))
    plt.tight_layout()
    return fig, ax    

def visualize_hillclimb(X, hillclimb):
    fig, ax = init_figure(X)
    permutations = hillclimb.optimize_explain(X)
    colors = get_colors([[cyclic_distance(X[p], hillclimb.dist) for p in permutations]], False)[0]
    coords = get_coords([X[p] for p in permutations])
    plot = ax.plot(coords[0][0], coords[0][1], color=colors[0])[0]
    
    def animate(i):
        plot.set_color(colors[i])
        plot.set_data(*coords[i])
        return (plot,)
    
    return animation.FuncAnimation(fig, animate, frames=len(permutations), interval=100, blit=True)

def visualize_genetic(X, genetic):
    fig, ax = init_figure(X)
    populations = genetic.optimize_explain(X)
    distances = [[cyclic_distance(X[p], genetic.dist) for p in populations[i]] for i in range(len(populations))]
    colors = get_colors(distances)
    coords = get_coords([X[p] for p in populations[0]])
    plots = [ax.plot(x, y, color=c)[0] for (x, y), c in zip(coords, colors[0])]
    best_id = np.argmin(distances[0])
    best_plot = ax.plot(*coords[best_id], color=(0.3, 0.3, 1, 0.9), linestyle="-.")[0]
    
    def animate(i):
        coords = get_coords([X[p] for p in populations[i]])
        for p, (x, y), c in zip(plots, coords, colors[i]):
            p.set_color(c)
            p.set_data(x, y)
        best_id = np.argmin(distances[i])
        best_plot.set_data(*coords[best_id])
        return plots + [best_plot]
    
    return animation.FuncAnimation(fig, animate, frames=len(populations), interval=100, blit=True)

In [3]:
def synthetic_points(count=25, dims=2):
    return np.random.randint(40, size=(count, dims))

X = synthetic_points()

### Задание 1 (1 балл)
Для начала научимся считать расстояния, которые курьерам придется преодолеть. Если бы они доставляли еду в пустыне, то им бы было достаточно считать евклидовы расстояния, т.к. можно просто идти напрямик. Однако курьеры доставляют еду только в городе, и двигаться могут только вдоль улиц. Поэтому нам понадобится манъэттенское расстояние.

#### Функции
`cyclic_distance(points, dist)` - возвращает суммарное расстояние пройденное курьером по циклу из точек `points` посчитанное при помощи функции `dist`.

`l2_distance(p1, p2)` - возвраает евклидово расстояние между точками.

`l1_distance(p1, p2)` - возвращает манхэттенское расстояние между точками.

In [4]:
def cyclic_distance(points, dist):
    return np.sum([dist(points[i - 1], points[i]) for i in range(len(points))])

def l2_distance(p1, p2):
    return np.linalg.norm(p1 - p2)

def l1_distance(p1, p2):
    return np.sum(np.abs(p1 - p2))

### Задание 2 (3 балла)
Курьер получил карту расположения заказов на карте. Ему хочется побыстрее обойти все точки. Для этого он хочет воспользоваться методом HillClimb, а вам предстоит ему в этом помочь. Реализуйте данный метод.

#### Методы
`optimize_explain(X)` - возвращает список из перестановок точек `X`, полученных в процессе оптимизации. Перестановка должна попадать в список после каждого изменения. Метод должен совершить не больше `max_iterations` итераций, на каждой из них необходимо рассмотреть все возможные пары индексов.

#### Параметры конструктора
`max_iterations` - максимальное количество итераций в методе `optimize_explain`

`dist` - функция расстояния

In [5]:
class HillClimb:
    def __init__(self, max_iterations, dist):
        self.max_iterations = max_iterations
        self.dist = dist # Do not change
    
    def optimize(self, X):
        return self.optimize_explain(X)[-1]
    
    def optimize_explain(self, X):
        permutations = []
        current_dist = cyclic_distance(X, self.dist)
        self.X = X
        i = 0

        return self.__next_permutation(permutations, np.arange(len(self.X)), current_dist, i)
    
    def __next_permutation(self, permutations, current_perm, current_dist, iteration):
        if iteration > self.max_iterations:
            return permutatuions

        best_perm = current_perm
        best_dist = current_dist
        for i in range(len(current_perm)):
            for j in range(i + 1, len(current_perm)):
                new_dist, new_perm = self.__new_permutation(current_perm, current_dist, i, j)

                if new_dist < best_dist:
                    best_dist = new_dist
                    best_perm = new_perm

                    permutations.append(new_perm)
                    return self.__next_permutation(permutations, best_perm, best_dist, iteration + 1)
        
        return permutations

    def __new_permutation(self, current_perm, current_dist, i, j):
        new_perm = current_perm.copy()
        new_perm[j], new_perm[i] = current_perm[i], current_perm[j]

        old_bound_sum = self.__calculate_bounds(current_perm, i, j)
        new_bound_sum = self.__calculate_bounds(new_perm, i, j)

        new_dist = current_dist - old_bound_sum + new_bound_sum

        return new_dist, new_perm

    def __calculate_bounds(self, permutation, i, j):
        l_in, l_out, r_in, r_out = 0, 0, 0, 0
        if i < j:
            l_in = self.dist(self.X[permutation][i], self.X[permutation][i + 1])
            r_in = self.dist(self.X[permutation][j], self.X[permutation][j - 1])
        
        if i > 0:
            l_out = self.dist(self.X[permutation][i - 1], self.X[permutation][i])

        if j < len(permutation) - 1:
            r_out = self.dist(self.X[permutation][j + 1], self.X[permutation][j])

        return l_in + l_out + r_in + r_out

In [6]:
hc = HillClimb(100, l1_distance)

print('X: ', cyclic_distance(X, l1_distance))
print('Hill Climb: ', cyclic_distance(X[hc.optimize(X)], l1_distance))

X:  746
Hill Climb:  312


In [ ]:
hc = HillClimb(100, l1_distance)
HTML(visualize_hillclimb(X, hc).to_jshtml())

### Задание 3 (4 балла)
Курьерское дело растет, теперь между городами блуждает большое количество курьеров, а их профессия вместе с известным маршрутом передается из поколение в поколение. Чем быстрее курьер способен обойти города - тем больше вероятность того, что он заработает себе на безоблачную старость и передаст свое дело потомкам. Симулируйте эволюцию курьеров, реализовав генетический алгоритм.

#### Методы
`optimize(X)` - выбирает лучшую перестановку из результатов `optimize_explain(X)`

`optimize_explain(X)` - возвращает список длины `iterations` популяций размера `population` перестановок точек `X`, полученных в процессе оптимизации. На каждом шаге алгоритм должен оставлять только `survivors` выживших, следующая популяция должна быть сгенерирована из них

In [8]:
class Genetic:
    def __init__(self, iterations, population, survivors, distance):
        self.pop_size = population
        self.surv_size = survivors
        self.dist = distance
        self.iters = iterations
    
    def optimize(self, X):
        population = self.optimize_explain(X)[-1]
        fitness = self.__calculate_fitness(population)

        best_idx = np.argmin(fitness)
        return population[best_idx]
    
    def optimize_explain(self, X):
        self.X = X
        population = self.__initialize()
        fitness = self.__calculate_fitness(population)

        result = []

        for iter in range(self.iters):
            survivors = self.__extinction(population, fitness, self.surv_size)
            
            population = self.__crossingover(survivors, self.pop_size)
            fitness = self.__calculate_fitness(population)

            population, fitness = self.__mutate_population(population, fitness)

            result.append(population)

        return result


    def __calculate_fitness(self, population):
        fitness = []

        for agent in population:
            fitness.append(cyclic_distance(self.X[agent], self.dist))

        return np.array(fitness)

    def __initialize(self):
        population = []

        first_perm = np.arange(len(self.X))

        for i in range(self.pop_size):
            current_pop = np.random.permutation(first_perm)
            population.append(current_pop)

        return np.array(population)

    def __mutate_population(self, population, fitness, mutation_rate=0.25):
        probs = fitness / np.sum(fitness)

        mutate_number = int(len(population) * mutation_rate)
        mutation_idx = np.random.choice(np.arange(len(population)), size=mutate_number, p=probs)

        for idx in mutation_idx:
            population[idx], fitness[idx] = self.__mutate(population[idx], fitness[idx])

        return population, fitness

    def __mutate(self, agent, agent_fitness):
        N = len(agent)
        i, j = np.random.randint(N, size=2)

        new_agent_fitness, new_agent = self.__new_permutation(agent, agent_fitness, i, j)

        return new_agent, new_agent_fitness

    def __crossingover(self, survivors, pop_size):
        population = []
        
        all_pairs = np.array(list(permutations(np.arange(len(survivors)), 2)))
        idx = np.random.choice(np.arange(len(all_pairs)), size = pop_size)
        all_pairs = all_pairs[idx]

        for i, j in all_pairs:
            new_agent = self.__crossingover_pair(survivors[i], survivors[j])
            population.append(new_agent)

        return np.array(population)
    
    def __crossingover_pair(self, first, second):
        new_agent = -np.ones_like(first)

        first_n = len(new_agent) // 2
        idx_first = np.random.randint(first_n)
        new_agent[:first_n] = first[idx_first: idx_first + first_n]

        j = first_n
        for value in second:
            if value not in new_agent:
                new_agent[j] = value
                j += 1

                if j == len(new_agent):
                    break

        return new_agent

    def __extinction(self, population, fitness, surv_size):
        sorted_fitness_idx = np.argsort(fitness)

        new_population = population[sorted_fitness_idx][:surv_size]

        return new_population

    def __new_permutation(self, current_perm, current_dist, i, j):
        new_perm = current_perm.copy()
        new_perm[j], new_perm[i] = current_perm[i], current_perm[j]

        old_bound_sum = self.__calculate_bounds(current_perm, i, j)
        new_bound_sum = self.__calculate_bounds(new_perm, i, j)

        new_dist = current_dist - old_bound_sum + new_bound_sum

        return new_dist, new_perm

    def __calculate_bounds(self, permutation, i, j):
        l_in, l_out, r_in, r_out = 0, 0, 0, 0
        if i < j:
            l_in = self.dist(self.X[permutation][i], self.X[permutation][i + 1])
            r_in = self.dist(self.X[permutation][j], self.X[permutation][j - 1])
        
        if i > 0:
            l_out = self.dist(self.X[permutation][i - 1], self.X[permutation][i])

        if j < len(permutation) - 1:
            r_out = self.dist(self.X[permutation][j + 1], self.X[permutation][j])

        return l_in + l_out + r_in + r_out

In [9]:
gen = Genetic(200, 100, 20, l1_distance)

print('X: ', cyclic_distance(X, l1_distance))
print('Genetic: ', cyclic_distance(X[gen.optimize(X)], l1_distance))
print('Hill Climb: ', cyclic_distance(X[hc.optimize(X)], l1_distance))

X:  746
Genetic:  248
Hill Climb:  312


In [ ]:
gen = Genetic(200, 100, 20, l1_distance)
HTML(visualize_genetic(X, gen).to_jshtml())